## Python 1 Project
#### IE MBD Apr 2024
#### Group 4

In [15]:
import pandas as pd

In [16]:
def read(path: str = None, db_config: dict = None):
    """
    This function should read a filepath/DB and return a dataframe
    """

    if db_config == None:
        return pd.read_csv(path)
    elif path == None:
        # Database Logic
        return None
    else:
        raise TypeError("What should we read???")

In [17]:
def processFiles(paths: dict = None, dbConfig: dict= None) -> tuple:
    """ 
    Read files from the relevant paths and use functions above to clean
    """

    retVal = []
    if paths != None:
        for path in paths['lst']:
            raw_df = read(paths['path_root']+path)
            raw_df.dropna(axis=1, inplace=True)
            retVal.append(raw_df)
    
    return retVal



In [26]:
def joinFrames(df_list: list) -> pd.DataFrame:
    """
    Join DF's together
    """
    retVal = 1
    for df in df_list:
        if isinstance(retVal, int):
            retVal = df
        else:
            if 'date' in df.columns:
                if 'date' in retVal.columns:
                    retVal = retVal.merge(df, on=["location_key", "date"], how="outer")
                    continue

            retVal = retVal.merge(df, on="location_key", how="outer")

    return retVal

In [19]:
def write(df: pd.DataFrame, dbConfig: dict = None) -> pd.DataFrame:
    """
    Write to CSV/DB
    """

In [20]:
def plot(df: pd.DataFrame) -> None:
    """
    Plot relevant data from the DF
    """

In [28]:
def main() -> None:

    files_config = {
        'lst': ["demographics", "epidemiology", "health", "hospitalizations", "index", "vaccinations"],
        'path_root': "./data/"
    }
    clean_dfs = processFiles(files_config)
    merged_dfs = joinFrames(clean_dfs)
    print(merged_dfs.info)
    write(merged_dfs)

main()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3476734 entries, 0 to 3476733
Data columns (total 13 columns):
 #   Column                               Dtype  
---  ------                               -----  
 0   location_key                         object 
 1   population                           float64
 2   date                                 object 
 3   cumulative_confirmed                 float64
 4   life_expectancy                      float64
 5   country_code                         object 
 6   country_name                         object 
 7   subregion1_code                      object 
 8   subregion1_name                      object 
 9   iso_3166_1_alpha_2                   object 
 10  iso_3166_1_alpha_3                   object 
 11  aggregation_level                    float64
 12  cumulative_persons_fully_vaccinated  float64
dtypes: float64(5), object(8)
memory usage: 371.4+ MB
None
